### Analysis of 3-D materials

This notebook assumes that the pre-processing has already been executed and the mesh is generated in xml
The tutorial for that is in meshes/gmsh/gmsh_notebook

By default .vtk files are not exported. If stress or displacement .vtk files are needed make sure to activate them from the epilysis function in the numerical analysis

In [ ]:
import sys
sys.path.insert(3,"../pre-processing/3D")
sys.path.insert(1,"../numerical_analysis")
sys.path.insert(2,"../analytical")
sys.path.insert(5,"../post-processing")
import numpy as np
from dolfin import *
import matplotlib as plt
import scipy.linalg as sc
import epilysis3D_functions as ep3f
import epilysis_3D
import store
import post_3D

#### Spherical Inclusions
Since the tutorial directory is one level above the base:

In [ ]:
meshes_location = "../meshes"
paraview_location = "../paraview"
results_location = "../results"

In [ ]:
name = "bulit_in"
store.save_name(name,results_location) #save the name for the post-processing!

The code is currently working with a cubic domain of length equals to one. It can be very easily modified to accept any length but no other length was needed.

In [ ]:
# mesh = Mesh(meshes_location+"/gmsh/" + name + ".xml")
# subdomains = MeshFunction("size_t", mesh, meshes_location+"/gmsh/" + name + "_physical_region.xml")
# store.save_mesh(name,mesh,subdomains,meshes_location,paraview_location)

# a=1 #length of cube in x
# b=1 #length of cube in y
# c=1 #length of cube in z
# vertices = np.array([[0, 0.,0.],#0
#                      [a, 0.,0.],#1
#                      [a,0.,c],#2
#                      [0.,0.,c],#3   Vertices of the cube
#                      [0.,b,c],#4    
#                      [0.,b,0.],#5
#                      [a,b,0.],#6
#                      [a,b,c]])#7

# vol = a*b*c #Volume of the cube

In [ ]:
mesh = Mesh(meshes_location+"/gmsh/" + name + ".xml")
subdomains = MeshFunction("size_t", mesh, meshes_location+"/gmsh/" + name + "_physical_region.xml")
store.save_mesh(name,mesh,subdomains,meshes_location,paraview_location)

xmin = mesh.coordinates()[:, 0].min()
ymin = mesh.coordinates()[:, 1].min()
zmin = mesh.coordinates()[:, 2].min()

xmax = mesh.coordinates()[:, 0].max()
ymax = mesh.coordinates()[:, 1].max()
zmax = mesh.coordinates()[:, 2].max()
a= xmax - xmin
b= ymax - ymin
c= zmax - zmin
vertices = np.array([[xmin,ymin,zmin],#0
                     [xmax, ymin,zmin],#1
                     [xmax,ymin,zmax],#2
                     [xmin,ymin,zmax],#3
                     [xmin,ymax,zmax],#4
                     [xmin,ymax,zmin],#5
                     [xmax,ymax,zmin],#6
                     [xmax,ymax,zmax]])#7
vol = a*b*c

In [ ]:
print (vertices)

In [ ]:
E = np.array([100.,100.],dtype = np.longlong) #Young moduli of the two phases
nu = np.array([0.2,0.2],dtype = np.longlong) #Poisson ratio's of the two phases

In [ ]:
print(epilysis_3D.calculate_Volume_fraction(mesh,subdomains))

In [ ]:
Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz = ep3f.ortho_from_iso(E,nu) #Convert iso to ortho format
store.save_E_nu_iso(name,E,nu,results_location) #save E,nu for the post-processing!
#-------------------------------------------------------------------------
print("Setting the problem !!!")
dx,w,Eps,F,a,L = epilysis_3D.fe_problem(mesh,subdomains,vertices,Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz) #set up the problem
print("Solving the equation !!!")
C_guess = epilysis_3D.calculate_moduli(name,mesh,vol,dx,w,Eps,F,a,L,Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz,meshes_location,paraview_location,results_location)#run the numerical analysis

In [ ]:
print(np.round(C_guess,3))

In [ ]:
K_Voigt,K_Reuss,K_Mori_Tanaka,K_num,G_Voigt,G_Reuss,G_Mori_Tanaka,G_num = post_3D.bounds(name,mesh,subdomains,E,nu,C_guess,results_location=results_location)

In [ ]:
print ("K from numerical analysis = ", K_num)
print ("K from M_T = ", K_Mori_Tanaka)
print("error = ", (abs(K_num - K_Mori_Tanaka))/K_Mori_Tanaka)

#### CT Scan (4 phase materieal) - randomly distributed phases

In [ ]:
name = "test_CT_scan"
store.save_name(name,results_location) #save the name for the post-processing!

In [ ]:
mesh = Mesh(meshes_location+"/gmsh/" + name + ".xml")
subdomains = MeshFunction("size_t", mesh, meshes_location+"/gmsh/" + name + "_physical_region.xml")
store.save_mesh(name,mesh,subdomains,meshes_location,paraview_location)

a=1 #length of cube in x
b=1 #length of cube in y
c=1 #length of cube in z
vertices = np.array([[0, 0.,0.],#0
                     [a, 0.,0.],#1
                     [a,0.,c],#2
                     [0.,0.,c],#3   Vertices of the cube
                     [0.,b,c],#4    
                     [0.,b,0.],#5
                     [a,b,0.],#6
                     [a,b,c]])#7

vol = a*b*c #Volume of the cube

In [ ]:
E = np.array([1000.,50.,500,10],dtype = np.longlong) #Young moduli of the two phases - 4 phases
nu = np.array([0.2,0.35,0.15,0.3],dtype = np.longlong) #Poisson ratio's of the two phases - 4 phases

In [ ]:
Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz = ep3f.ortho_from_iso(E,nu) #Convert iso to ortho format
store.save_E_nu_iso(name,E,nu,results_location) #save E,nu for the post-processing!
#----------------------------------------------------------------------------------------------------------------
dx,w,Eps,F,a,L = epilysis_3D.fe_problem(mesh,subdomains,vertices,Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz) #set up the problem
C_guess = epilysis_3D.calculate_moduli(name,mesh,vol,dx,w,Eps,F,a,L,Exx,Eyy,Ezz,Gxy,Gzx,Gyz,nuxy,nuzx,nuyz,meshes_location,paraview_location,results_location)#run the numerical analysis

In [ ]:
print(np.round(C_guess,3))